This notebook demonstrates unit conversion in the profile viewer

In [1]:
import numpy as np
from astropy.wcs import WCS
import glue_jupyter as gj
from glue.core import Data
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink

Set up two datasets with x-axes that use different spectral coordinates (which get auto-converted because of the WCSLink) and y-axes that use different temperature units (just to show how to add custom equivalencies)

In [2]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['WAVE']
wcs1.wcs.crval = [10]
wcs1.wcs.cdelt = [3]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['cm']

d1 = Data(temp1=np.random.random(10))
d1.get_component('temp1').units = 'deg_C'
d1.coords = wcs1

In [3]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1000]
wcs1.wcs.cdelt = [1000]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['MHz']

d2 = Data(temp2=np.random.random(10) + 273)
d2.get_component('temp2').units = 'K'
d2.coords = wcs1

Set up a custom unit conversion class with equivalencies:

In [4]:
from astropy import units as u
from glue.core.units import unit_converter

@unit_converter('custom-astropy')
class UnitConverterWithTemperature:

    def equivalent_units(self, data, cid, units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return map(u.Unit(units).find_equivalent_units(include_prefix_units=True, equivalencies=equivalencies), str)

    def to_unit(self, data, cid, values, original_units, target_units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return (values * u.Unit(original_units)).to_value(target_units, equivalencies=equivalencies)

Use the converter class defined above

In [5]:
from glue.config import settings
settings.UNIT_CONVERTER = 'custom-astropy'

In [6]:
app = gj.jglue(spectrum1=d1, spectrum2=d2)

/home/tom/Code/glue/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')
/home/tom/Code/glue/glue/viewers/common/qt/tool.py:5: UserWarning: glue.viewers.common.qt.tool is deprecated, use glue.viewers.common.tool instead
  warnings.warn('glue.viewers.common.qt.tool is deprecated, use '


In [7]:
app.output

Output()

In [8]:
app.data_collection.add_link(WCSLink(d1, d2))

In [9]:
app.data_collection

DataCollection (2 data sets)
	  0: spectrum1
	  1: spectrum2

In [10]:
s = app.profile1d(data=d1)
s.add_data(d2)

equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False


LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False


True

In [11]:
s.state.x_display_unit = 'cm'
s.state.y_display_unit = 'K'

equiv None wave False
equiv None wave False
equiv None wave False
equiv None wave False
equiv [(Unit("K"), Unit("deg_C"), <function temperature.<locals>.<lambda> at 0x7f13d0f00160>, <function temperature.<locals>.<lambda> at 0x7f13d0f00670>), (Unit("deg_C"), Unit("deg_F"), <function temperature.<locals>.<lambda> at 0x7f13d0f00700>, <function temperature.<locals>.<lambda> at 0x7f13d0f00790>), (Unit("K"), Unit("deg_F"), <function temperature.<locals>.<lambda> at 0x7f13d0f00820>, <function temperature.<locals>.<lambda> at 0x7f13d0f008b0>), (Unit("deg_R"), Unit("deg_F"), <function temperature.<locals>.<lambda> at 0x7f13d0f00940>, <function temperature.<locals>.<lambda> at 0x7f13d0f009d0>), (Unit("deg_R"), Unit("deg_C"), <function temperature.<locals>.<lambda> at 0x7f13d0f00a60>, <function temperature.<locals>.<lambda> at 0x7f13d0f00b80>), (Unit("deg_R"), Unit("K"), <function temperature.<locals>.<lambda> at 0x7f13d0f00af0>, <function temperature.<locals>.<lambda> at 0x7f13d0f00c10>)] temp1